## Import modules

In [ ]:
import numpy as np
import pandas as pd
import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
from matplotlib.pyplot import plot

style.use("fivethirtyeight")
%matplotlib inline
%config InlineBackend.figure_format = "retina"

## Import data

In [ ]:
df = pd.read_csv("/kaggle/input/avocado-prices/avocado.csv")

In [ ]:
data_size_mb = df.memory_usage().sum() / 1024 / 1024
print("Data memory size: %.2f MB" % data_size_mb)

## Data exploration

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending = False)

## Data visualization

### Price distribution

In [ ]:
plt.figure(figsize = (12,6))
sns.distplot(df["AveragePrice"])
plt.title("Price distribution")
plt.show()

### Average price of conventional vs. organic

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["Month"] = df["Date"].dt.month

plt.figure(figsize = (12,6))
sns.lineplot(data = df, x = "Month", y = "AveragePrice", hue = "type")
plt.title("Average price: conventional vs. organic (per month)")
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
sns.lineplot(data = df, x = "year", y = "AveragePrice", hue = "type")
plt.title("Average price: conventional vs. organic (per year)")
plt.show()

### Volume by month and year

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(data = df, x = "Month", y = "Total Volume", hue = "type")
plt.title("Total volume by month")
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(data = df, x = "year", y = "Total Volume", hue = "type")
plt.title("Total volume by year")
plt.show()

### Volume by region

In [ ]:
plt.figure(figsize = (12,6))

sns.barplot(data = df[df.region != "TotalUS"], 
            x = "region", 
            y = "Total Volume", 
            hue = "type", 
            ci = None)

plt.title("Total volume by region")
plt.xticks(rotation = 90)
plt.show()

### Comparison of bagsize

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(data = df, x = "Month", y = "Small Bags", color = "blue", label = "Small Bags")
sns.barplot(data = df, x = "Month", y = "Large Bags", color = "orange", label = "Large Bags")
sns.barplot(data = df, x = "Month", y = "XLarge Bags", color = "green", label = "XLarge Bags")
plt.title("Bag sales per month")
plt.ylabel("Bags")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
sns.barplot(data = df, x = "year", y = "Small Bags", color = "blue", label = "Small Bags")
sns.barplot(data = df, x = "year", y = "Large Bags", color = "orange", label = "Large Bags")
sns.barplot(data = df, x = "year", y = "XLarge Bags", color = "green", label = "XLarge Bags")
plt.title("Bag sales per year")
plt.ylabel("Bags")
plt.legend()
plt.show()

## Machine Learning

In [ ]:
df = df.drop(["Unnamed: 0", "Date"], axis = 1)
df["Month"] = df["Month"].astype("int32")

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

In [ ]:
X = df.drop("AveragePrice", axis = 1)
Y = df["AveragePrice"]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    random_state = 0, 
                                                    test_size = 0.25)

### XGBoost

In [ ]:
clf_xgb = XGBRegressor(n_estimators = 700, max_depth = 5)
clf_xgb.fit(X_train, Y_train)
print(round(clf_xgb.score(X_test, Y_test), 4))

### Random Forest

In [ ]:
clf_rf = RandomForestRegressor(n_estimators = 300)
clf_rf.fit(X_train, Y_train)
print(round(clf_rf.score(X_test, Y_test), 4))

### Stacking

In [ ]:
vote = VotingRegressor(estimators = [("XGBoost", clf_xgb), 
                                     ("RandomForest", clf_rf)])

vote.fit(X_train, Y_train)
print(round(vote.score(X_test, Y_test), 4))